# Introduction

Concatenates already-preprocessed VNP46A1 GeoTiff files that are spatially adjacent in the longitudinal direction and exports single GeoTiff files containing the concatenated data. Used in cases when a study area bounding box intersects two VNP46A1 grid cells (.e.g. `VNP46A1.A2020001.h30v05.001.2020004003738.h5` and `VNP46A1.A2020001.h31v05.001.2020004003738.h5` for raw files and `vnp46a1-a2020001-h30v05-001-2020004003738.tif` and `vnp46a1-a2020001-h31v05-001-2020004003841.tif` for already-preprocessed files).

This Notebook uses the following folder structure:

```
├── 01-code-scripts
│   ├── clip_vnp46a1.ipynb
│   ├── clip_vnp46a1.py
│   ├── concatenate_vnp46a1.ipynb
│   ├── concatenate_vnp46a1.py
│   ├── download_laads_order.ipynb
│   ├── download_laads_order.py
│   ├── preprocess_vnp46a1.ipynb
│   ├── preprocess_vnp46a1.py
│   └── viirs.py
├── 02-raw-data
├── 03-processed-data
├── 04-graphics-outputs
└── 05-papers-writings
```

Running the Notebook from the `01-code-scripts/` folder works by default. If the Notebook runs from a different folder, the paths in the environment setup section may have to be changed.

This notebook uses files that have alrady been preprocessed and saved to GeoTiff files.

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import warnings
import glob
import viirs

In [ ]:
# Set options
warnings.simplefilter("ignore")

In [ ]:
# Set working directory
os.chdir("..")

# User-Defined Variables

In [ ]:
# Define path to folder containing preprocessed VNP46A1 GeoTiff files
geotiff_input_folder = os.path.join(
    "03-processed-data", "raster", "south-korea", "vnp46a1-grid"
)

# Defne path to output folder to store concatenated, exported GeoTiff files
geotiff_output_folder = os.path.join(
    "03-processed-data", "raster", "south-korea", "vnp46a1-grid-concatenated"
)

# Set start date and end date for processing
start_date, end_date = "2020-01-01", "2020-04-09"

# Data Preprocessing

In [ ]:
# Concatenate and export adjacent images that have the same acquisition date
dates = viirs.create_date_range(start_date=start_date, end_date=end_date)
geotiff_files = glob.glob(os.path.join(geotiff_input_folder, "*.tif"))
concatenated_dates = 0
skipped_dates = 0
processed_dates = 0
total_dates = len(dates)
for date in dates:
    adjacent_images = []
    for file in geotiff_files:
        if date in viirs.extract_date_vnp46a1(geotiff_path=file):
            adjacent_images.append(file)
    adjacent_images_sorted = sorted(adjacent_images)
    if len(adjacent_images_sorted) == 2:
        viirs.concatenate_preprocessed_vnp46a1(
            west_geotiff_path=adjacent_images_sorted[0],
            east_geotiff_path=adjacent_images_sorted[1],
            output_folder=geotiff_output_folder,
        )
        concatenated_dates += 1
    else:
        skipped_dates += 1
    processed_dates += 1
    print(f"Processed dates: {processed_dates} of {total_dates}\n\n")

print(
    f"Concatenated dates: {concatenated_dates}, Skipped dates: {skipped_dates}"
)